In [ ]:
using Revise
using Plots
using Hungarian
using Gen
using Statistics

includet("./distribution_utils.jl")
includet("./model.jl")
includet("./visualizations.jl")
includet("./inference.jl")

In [ ]:
function visualize_importance_samples(traces, constraint_img, log_weights, scene_size)
    fig = plot(layout = grid(1,2),  size=(400, 400), legend=false)
    frame = channelview(RGB.(constraint_img))
    heatmap!(fig[2], frame, aspect_ratio=:equal, xlims=[0, scene_size + 1], ylims=[0, scene_size + 1])
    for t in 1:length(traces)
        trace = traces[t]
        choices = get_choices(trace)
        n_fireflies = choices[:init => :n_fireflies]
        xs = [choices[:init => :init_x => n] for n in 1:n_fireflies]
        ys = [choices[:init => :init_y => n] for n in 1:n_fireflies]
        color_opts = ["red", "green", "blue"]
        colors = [color_opts[choices[:init => :color => n]] for n in 1:n_fireflies]
        scatter!(fig[1], xs, ys, color=colors, markersize=5, alpha=log_weights[t] + 0.001, aspect_ratio=:equal,  xlims=[0, scene_size + 1], ylims=[0, scene_size + 1], yflip=true,
                label="Importance Samples")
    end

    return fig
end

function visualize_importance_resampling(trace, constraint_img)
    fig = plot(layout = grid(1,2), size=(400, 400), legend=false)
    frame = colorview(RGB, constraint_img)
    Plots.plot!(fig[2], frame, title="Frame", aspect_ratio=:equal, xlims=[0, scene_size + 1], ylims=[0, scene_size + 1])
    choices = get_choices(trace)
    n_fireflies = choices[:init => :n_fireflies]
    xs = [choices[:init => :init_x => n] for n in 1:n_fireflies]
    ys = [choices[:init => :init_y => n] for n in 1:n_fireflies]
    color_opts = ["red", "green", "blue"]
    colors = [color_opts[choices[:init => :color => n]] for n in 1:n_fireflies]
    scatter!(fig[1], xs, ys, color=colors, markersize=5, aspect_ratio=:equal,  xlims=[0, scene_size + 1], ylims=[0, scene_size + 1], yflip=true,
    label="Importance Samples")
    return fig
end

In [ ]:
function find_color_patches(image::Array{Float64, 3}, threshold::Float64, size_prior::Int, color_threshold::Float64)
    height, width = size(image, 2), size(image, 3)
    visited = zeros(Int, height, width)
    patches = []

    # Function to flood fill and track patch
    function flood_fill(y, x)
        stack = [(y, x)]
        patch = []
        initial_color = image[:, y, x]
        too_large = false

        while !isempty(stack)
            cy, cx = pop!(stack)
            if cy < 1 || cy > height || cx < 1 || cx > width || visited[cy, cx] == 1
                continue
            end

            pixel_color = image[:, cy, cx]
            
            # Luminance filter and color filter
            if maximum(pixel_color) <= threshold || norm(pixel_color - initial_color) > color_threshold
                continue
            end

            visited[cy, cx] = 1
            push!(patch, (cx, cy, argmax(pixel_color)))            

            for dy in -1:1, dx in -1:1
                if dy == 0 && dx == 0
                    continue
                end
                push!(stack, (cy + dy, cx + dx))
            end
        end

        if length(patch) > size_prior  
            too_large = true
        end
        
        return patch, too_large
    end

    # Function to split large patches using simple median split
    function split_patch(patch)
        xs, ys = [p[1] for p in patch], [p[2] for p in patch]
        mean_x, mean_y = mean(xs), mean(ys)
        cluster1, cluster2 = [], []
        
        for p in patch
            if p[1] < mean_x || p[2] < mean_y
                push!(cluster1, p)
            else
                push!(cluster2, p)
            end
        end

        return cluster1, cluster2
    end

    # Main loop to find patches
    for y in 1:height, x in 1:width
        if maximum(image[:, y, x]) > threshold && visited[y, x] == 0
            patch, too_large = flood_fill(y, x)
            if too_large
                cluster1, cluster2 = split_patch(patch)
                push!(patches, cluster1, cluster2)
            else
                push!(patches, patch)
            end
        end
    end

    return patches, length(patches)
end


@gen function feature_based_proposal(max_fireflies, img_array)
    _, _, scene_size = size(img_array)
    
    # Sum across color channels and reshape to 2D
    # intensity = dropdims(sum(img_array, dims=1), dims=1)
    
    threshold = 0.2 # Luminance threshold
    size_prior = 30 # size prior for splitting patches
    color_threshold = 0.7 # color matching threshold
    patch_info, num_clusters = find_color_patches(img_array, threshold, size_prior, color_threshold)
    n_fireflies = {:init => :n_fireflies} ~ uniform_discrete(num_clusters, num_clusters)
    for n in 1:num_clusters
        patch = patch_info[n]
        patch_index = uniform_discrete(1, length(patch))

        x_opts = ones(scene_size) .* 0.001
        y_opts = ones(scene_size) .* 0.001
        color_opts = ones(3) .* 0.001

        # Constrain initialization to area of patch
        x_opts[patch[patch_index][1]] = 1 
        y_opts[patch[patch_index][2]] = 1 
        color_opts[patch[patch_index][3]] = 1

        x_opts = x_opts ./ sum(x_opts)
        y_opts = y_opts ./ sum(y_opts)
        color_opts = color_opts ./ sum(color_opts)

        x = {:init => :init_x => n} ~ categorical(x_opts)
        y = {:init => :init_y => n} ~ categorical(y_opts)
        color = {:init => :color => n} ~ categorical(color_opts)
    end
end

In [ ]:
scene_size = 64
max_fireflies = 4
steps = 1

constraints = choicemap()

# Test case 1 - overlap different color
gt_constraints = choicemap()
gt_constraints[:init => :n_fireflies] = 2
gt_constraints[:states => 1 => :x => 1] = 10
gt_constraints[:states => 1 => :y => 1] = 10
gt_constraints[:init => :color => 1] = 1

gt_constraints[:states => 1 => :x => 2] = 12
gt_constraints[:states => 1 => :y => 2] = 12
gt_constraints[:init => :color => 2] = 2

gt_trace, w = generate(model, (scene_size, max_fireflies, steps,), gt_constraints)
choices = get_choices(gt_trace)

constraints = choicemap()
constraints[:observations => 1] = choices[:observations => 1]
true_x = [choices[:states => 1 => :x => n] for n in 1:choices[:init => :n_fireflies]]
true_y = [choices[:states => 1 => :y => n] for n in 1:choices[:init => :n_fireflies]]

tr, lml_est = importance_resampling(model, get_args(gt_trace), constraints, 
                                    feature_based_proposal, 
                                    (max_fireflies, choices[:observations => 1]), 10)

inferred_choices = get_choices(tr)
inf_xs = [inferred_choices[:states => 1 => :x => n] for n in 1:inferred_choices[:init => :n_fireflies]]
inf_ys = [inferred_choices[:states => 1 => :y => n] for n in 1:inferred_choices[:init => :n_fireflies]]
inf_color = [inferred_choices[:init => :color => n] for n in 1:inferred_choices[:init => :n_fireflies]]

println("Inferred n_fireflies: ", inferred_choices[:init => :n_fireflies])
println("GT n_fireflies: ", choices[:init => :n_fireflies])
fig = visualize_importance_resampling(tr, choices[:observations => 1])
scatter!(fig[1], true_x, true_y, color="black", markershape=:x, markersize=5, label="True Locations")
display(fig)


# Test case 2 - overlap same color
gt_constraints = choicemap()
gt_constraints[:init => :n_fireflies] = 2
gt_constraints[:states => 1 => :x => 1] = 10
gt_constraints[:states => 1 => :y => 1] = 10
gt_constraints[:init => :color => 1] = 1
gt_constraints[:states => 1 => :x => 2] = 12
gt_constraints[:states => 1 => :y => 2] = 12
gt_constraints[:init => :color => 2] = 1

gt_trace, w = generate(model, (scene_size, max_fireflies, steps,), gt_constraints)
choices = get_choices(gt_trace)

constraints = choicemap()
constraints[:observations => 1] = choices[:observations => 1]
true_x = [choices[:states => 1 => :x => n] for n in 1:choices[:init => :n_fireflies]]
true_y = [choices[:states => 1 => :y => n] for n in 1:choices[:init => :n_fireflies]]

tr, lml_est = importance_resampling(model, get_args(gt_trace), constraints, 
                                    feature_based_proposal, 
                                    (max_fireflies, choices[:observations => 1]), 10)

inferred_choices = get_choices(tr)
println("Inferred n_fireflies: ", inferred_choices[:init => :n_fireflies])
println("GT n_fireflies: ", choices[:init => :n_fireflies])
inf_xs = [inferred_choices[:states => 1 => :x => n] for n in 1:inferred_choices[:init => :n_fireflies]]
inf_ys = [inferred_choices[:states => 1 => :y => n] for n in 1:inferred_choices[:init => :n_fireflies]]
inf_color = [inferred_choices[:init => :color => n] for n in 1:inferred_choices[:init => :n_fireflies]]

fig = visualize_importance_resampling(tr, choices[:observations => 1])
scatter!(fig[1], true_x, true_y, color="black", markershape=:x, markersize=5, label="True Locations")
display(fig)